In [1]:
from impedance.models.circuits import CustomCircuit
from impedance.models.circuits.elements import element
import numpy as np
import scipy

In [2]:
@element(num_params=3, units=["Ohm", "Ohm", "F"],overwrite=True)
def TLMilinNew(p,f):
    omega = 2 * np.pi * np.array(f)
    R1, R2, Q = p[0], p[1], p[2]
    t12=R1/R2
    Rion1=R1
    Rion2=R2
    wclin1=1/(Rion1*Q)
    wclin2=1/(Rion2*Q)
    S1=np.sqrt(1j*omega/wclin1)
    S2=np.sqrt(1j*omega/wclin2)
    
    Z = -(2*1j*R1/S1)*(((scipy.special.yv(1,-2*1j*t12*S2/(t12-1)))*(scipy.special.jv(0,-2*1j*S1/(t12-1))))-((scipy.special.jv(1,-2*1j*t12*S2/(t12-1)))*(scipy.special.yv(0,-2*1j*S1/(t12-1)))))/(((scipy.special.jv(1,-2*1j*t12*S2/(t12-1)))*(scipy.special.yv(1,-2*1j*S1/(t12-1))))-((scipy.special.yv(1,-2*1j*t12*S2/(t12-1)))*(scipy.special.jv(1,-2*1j*S1/(t12-1)))))
    
    return Z



In [3]:
Rref=194.56
t1=6.4
R1=Rref*t1
t2=3.13
R2=Rref*t2
Q=6.61e-4
tlmilin=CustomCircuit(initial_guess=[1245.184,608.9728,6.61e-4],circuit='TLMilinNew_0')
tlmilin_guess=CustomCircuit(initial_guess=[1245,608,6.61e-4],circuit='TLMilinNew_0')

In [4]:
f_pred = np.logspace(3,-3,num=100)
tlmuni_pred = tlmilin.predict(f_pred)
print(tlmuni_pred)

[          nan           +nanj    0.        -0.00000000e+00j
           nan           +nanj           nan           +nanj
           inf           -infj -104.13039263-3.47101309e+01j
           nan           +nanj   -0.        +0.00000000e+00j
          -inf           -infj  -15.29491676+1.52949168e+01j
           nan           +nanj  -18.61987504+3.10331251e+01j
          -inf           -infj          -inf           +infj
           nan           +nanj          -inf           -infj
           inf           -infj    0.        +1.60369808e+02j
  -85.97964451+8.59796445e+01j           nan           +nanj
           inf           -infj           nan           +nanj
  -50.70985954+6.46987863e+01j   -0.        +0.00000000e+00j
  241.25799618-1.60838664e+02j  138.57989669-1.37549563e+02j
  149.66390702-1.47067480e+02j  160.91252548-1.55666785e+02j
  172.59074389-1.66651652e+02j  185.05049791-1.78240904e+02j
  198.3882904 -1.90589095e+02j  212.6641445 -2.03748581e+02j
  227.93525926-2.1779530

C:\Users\admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\impedance\models\circuits\circuits.py:145: UserWarning: Simulating circuit based on initial parameters
  warnings.warn("Simulating circuit based on initial parameters")
C:\Users\admin\AppData\Local\Temp\ipykernel_34312\2097421082.py:13: RuntimeWarning: divide by zero encountered in divide
  Z = -(2*1j*R1/S1)*(((scipy.special.yv(1,-2*1j*t12*S2/(t12-1)))*(scipy.special.jv(0,-2*1j*S1/(t12-1))))-((scipy.special.jv(1,-2*1j*t12*S2/(t12-1)))*(scipy.special.yv(0,-2*1j*S1/(t12-1)))))/(((scipy.special.jv(1,-2*1j*t12*S2/(t12-1)))*(scipy.special.yv(1,-2*1j*S1/(t12-1))))-((scipy.special.yv(1,-2*1j*t12*S2/(t12-1)))*(scipy.special.jv(1,-2*1j*S1/(t12-1)))))
C:\Users\admin\AppData\Local\Temp\ipykernel_34312\2097421082.py:13: RuntimeWarning: invalid value encountered in divide
  Z = -(2*1j*R1/S1)*(((scipy.special.yv(1,-2*1j*t12*S2/(t12-1)))*(scipy.special.jv(0,-2*1j*

In [5]:
fitted_ilin_on_ilin=tlmilin_guess.fit(f_pred,tlmuni_pred,weight_by_modulus=True)
print(fitted_ilin_on_ilin)

ValueError: array must not contain infs or NaNs